In [91]:
# Import packages
import pandas as pd
import numpy as np
import requests
import time
from datetime import date
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
from dateutil.relativedelta import relativedelta

In [92]:
# API SET UP

# coinmarketcap key
fp = '../0-admin/coinmarketcap_standard.txt'

with open(fp) as f:
    API_KEY = f.readlines()
    API_KEY = API_KEY[0].strip()
    
# Set up API
base_url = "https://pro-api.coinmarketcap.com"
headers = {'Accepts': 'application/json',
           'X-CMC_PRO_API_KEY': API_KEY}

# Test it is working
endpoint = '/v1/key/info'
final_url = base_url + endpoint
session = Session()
session.headers.update(headers)
r = session.get(final_url)
print(r.json())

{'status': {'timestamp': '2022-08-29T23:23:23.097Z', 'error_code': 0, 'error_message': None, 'elapsed': 4, 'credit_count': 0, 'notice': "You have used 300% of your plan's daily credit limit."}, 'data': {'plan': {'credit_limit_daily': 16666, 'credit_limit_daily_reset': 'In 16 hours, 48 minutes', 'credit_limit_daily_reset_timestamp': '2022-08-30T16:12:20.000Z', 'credit_limit_monthly': 500000, 'credit_limit_monthly_reset': 'In 28 days, 16 hours, 48 minutes', 'credit_limit_monthly_reset_timestamp': '2022-09-27T16:12:20.000Z', 'rate_limit_minute': 60}, 'usage': {'current_minute': {'requests_made': 1, 'requests_left': 59}, 'current_day': {'credits_used': 50024, 'credits_left': -33358}, 'current_month': {'credits_used': 50024, 'credits_left': 449976}}}}


In [93]:
# OBTAIN PRICE VOLUME AND MCAP DATA

# Note: this call takes a few minutes.
# Obtain price, volume, and mcap data
token_dfs = []
btc_eth_cmc_ids = [1, 1027]

# Specify the dates to obtain
first_date = '2017-11-30'
last_date  = '2022-09-05'
dates      = pd.date_range(first_date,
                           last_date,
                           freq='M').strftime("%Y-%m-%d").tolist()

# Repeat for all 1 month intervals for BTC and ETH
for token_id in btc_eth_cmc_ids:
    for i in range(0,len(dates)-1):
        # specify dates
        start_date = dates[i]
        end_date   = dates[i+1]
        print(end_date)

        # Set up the call
        endpoint = '/v1/cryptocurrency/quotes/historical'
        final_url = base_url+endpoint
        token_id = str(token_id)
        parameters = {'id': token_id,
                      'time_start': start_date,
                      'time_end': end_date,
                      'count': 10000,
                      'interval': '5m',
                      'convert': 'USD'}

        # Make the call
        try:
            response = session.get(final_url, params=parameters)
            r_json = json.loads(response.text)
            if (r_json['status']['error_message'] == None):
                data = r_json['data']
            elif (r_json['status']['error_message'][:29] == 'Search query is out of range.'):
                continue # skip if the data is not in the range of interest
            else:
                assert(1==0),'json has error'

        except (ConnectionError, Timeout, TooManyRedirects) as e:
            print(e)
            print('\n')

        # Add the cleaned up data as a data frame
        token_quote_dict_list = []
        for quote in data['quotes']:
            new_dict = {}
            new_dict['date']           = quote['quote']['USD']['timestamp']
            new_dict['usd_per_token']  = quote['quote']['USD']['price']
            new_dict['usd_volume_24h'] = quote['quote']['USD']['volume_24h']
            new_dict['usd_mcap']       = quote['quote']['USD']['market_cap']
            token_quote_dict_list.append(new_dict)

        token_df = pd.DataFrame(token_quote_dict_list)
        token_df['cmc_id'] = data['id']
        token_dfs.append(token_df)

        # space out calls and track
        time.sleep(1)

# clean up the data
df = pd.concat(token_dfs)
df = df[['date', 'cmc_id', 'usd_per_token', 'usd_volume_24h', 'usd_mcap']]
df['date'] = pd.to_datetime(df.date).dt.tz_localize(None)
df['date'] = df.date.dt.ceil('5min')
df = df.dropna()
df = df.drop_duplicates(subset=['date', 'cmc_id'])
dates_df = pd.concat((pd.DataFrame(data={'date': pd.date_range(first_date, last_date, freq='5min'),
                                        'cmc_id': 1}),
                      pd.DataFrame(data={'date': pd.date_range(first_date, last_date, freq='5min'),
                                        'cmc_id': 1027})))
df = df.merge(dates_df,
              on=['date', 'cmc_id'],
              how='outer',
              validate='one_to_one')
df = df.sort_values(by=['cmc_id', 'date'], ignore_index=True)
df = df.interpolate(method='ffill', limit_area='inside')
df = df.dropna()
assert(0==df.isnull().sum().sum())
df = df.sort_values(by=['date', 'cmc_id'], ignore_index=True)

2017-12-31


AssertionError: json has error

In [ ]:
df = df[~((df.date.dt.year==2017)&(df.date.dt.month==11))]

In [ ]:
# OBTAIN GLOBAL COINMARKETCAP DATA
# NOTE: this takes about 5 minutes
macro_dfs = []

# Repeat for all 1 month intervals for BTC and ETH
for i in range(0,len(dates)-1):
    # specify dates
    start_date = dates[i]
    end_date   = dates[i+1]

    # Set up the call
    endpoint = '/v1/global-metrics/quotes/historical'
    final_url = base_url+endpoint
    parameters = {'time_start': start_date,
                  'time_end': end_date,
                  'count': 2,
                  'interval': '5m',
                  'convert': 'USD',
                  'aux': 'btc_dominance,total_volume_24h,altcoin_market_cap'}

    response = session.get(final_url, params=parameters)
    r_json = json.loads(response.text)

    # Initialize dictionary for the data
    cmc_macro_dict = {'date': [],
                      'total_volume_24h': [],
                      'altcoin_market_cap': [],
                      'btc_dominance': []}

    # Convert JSON into dictionary
    for token in r_json['data']['quotes']:
        cmc_macro_dict['date'].append(token['timestamp'])
        cmc_macro_dict['total_volume_24h'].append(token['quote']['USD']['total_volume_24h'])
        cmc_macro_dict['altcoin_market_cap'].append(token['quote']['USD']['altcoin_market_cap'])
        cmc_macro_dict['btc_dominance'].append(token['btc_dominance'])

    # Cut out duplicated dates in macro data
    macro_df = pd.DataFrame(cmc_macro_dict)
    macro_df = macro_df[macro_df.index.isin(macro_df['date'].drop_duplicates().index)]

    # Append
    macro_dfs.append(macro_df)

    # space out calls and track
    time.sleep(1)

# Clean up the macro dataframe to have all study period dates and interpolate missing dates
macro_df = pd.concat(macro_dfs)
macro_df['date'] = pd.to_datetime(macro_df.date).dt.tz_localize(None)
macro_df['date'] = macro_df.date.dt.ceil('5min')
macro_df = macro_df.drop_duplicates(subset='date')
df = df.dropna()
dates_df = pd.DataFrame(data={'date': pd.date_range(first_date, last_date, freq='5min')})
macro_df = macro_df.merge(dates_df,
                          on='date',
                          how='outer',
                          validate='one_to_one')

macro_df = macro_df.sort_values(by='date', ignore_index=True)
macro_df = macro_df.interpolate(method='ffill', limit_area='inside')
macro_df = macro_df.dropna()
assert(0==macro_df.isnull().sum().sum())
macro_df = macro_df.sort_values(by='date', ignore_index=True)
macro_df['date'] += pd.Timedelta(minutes=5) # DATA HAS 5 MIN OF LATENCY SO WE HAVE TO LAG IT BY 5 MIN

In [ ]:
macro_df = macro_df[~((macro_df.date.dt.year==2017)&(macro_df.date.dt.month==11))]

In [ ]:
# TODO FIX BELOW CODE!

# READ IN THE OLD DATA
old_df        = pd.read_csv('../3-data/raw/cmc_price_vol_mcap_btceth_5min-20171201_20220824.csv')
old_macro_df  = pd.read_csv('../3-data/raw/cmc_macro_timeseries_btceth_5min-20171201_20220824.csv')

# COMBINE DATA

# drop dates in the existing data from the new data pulled
last_date_old_main_data  = np.max(old_df.date.values)
df                       = df[df.date>last_date_old_main_data]
last_date_old_macro_data = np.max(old_macro_df.date.values)
macro_df                 = macro_df[macro_df.date>last_date_old_macro_data]

# put data together
df = pd.concat((old_df, df)).reset_index(drop=True)
macro_df = pd.concat((old_macro_df, macro_df)).reset_index(drop=True)

# sort
df            = df.sort_values(by=['date', 'cmc_id'])
macro_df      = macro_df.sort_values(by=['date'])


In [234]:
## SAVE DATA

df.to_csv('../3-data/raw/cmc_price_vol_mcap_btceth_5min.csv', index=False)
macro_df.to_csv('../3-data/raw/cmc_macro_timeseries_btceth_5min.csv', index=False)

,date,total_volume_24h,altcoin_market_cap,btc_dominance
0,2017-11-30T00:07:00.000Z,2.427597e+10,1.296200e+11,56.113804
1,2017-11-30T00:12:00.000Z,2.442784e+10,1.305865e+11,56.026665
2,2017-11-30T00:17:00.000Z,2.470249e+10,1.319481e+11,56.087906
3,2017-11-30T00:22:00.000Z,2.500412e+10,1.338523e+11,55.931969
4,2017-11-30T00:27:00.000Z,2.511601e+10,1.341620e+11,55.986496
...,...,...,...,...
7478,2022-08-25T23:20:00.000Z,6.434394e+10,6.288437e+11,39.616971
7479,2022-08-25T23:25:00.000Z,6.439341e+10,6.293038e+11,39.611672
7480,2022-08-25T23:30:00.000Z,6.431970e+10,6.298418e+11,39.618041
7481,2022-08-25T23:35:00.000Z,6.429181e+10,6.298992e+11,39.619576


In [ ]:
# TEMP TODO

# DROP NOV 2017 DATA
# DROP AUGUST 29 DATA FROM TODAY

# GO DELETE THAT OLD FILE FOR BOTH SO I DONT CARRY AROUND THE DATA

In [ ]:
# SAVE DATA TO NEW DATES IF IT IS A NEW MONTH

df.to_csv('../3-data/raw/cmc_price_vol_mcap_btceth_5min-20171201_20220828.csv', index=False)
macro_df.to_csv('../3-data/raw/cmc_macro_timeseries_btceth_5min-20171201_20220828.csv', index=False)
